In [ ]:
1.引入torchvision库

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model2 = torchvision.models.alexnet(pretrained=False)
model2.classifier[6] = torch.nn.Linear(model2.classifier[6].in_features, 2)

In [ ]:
2.部署巡线+避障模型和停车模型

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))
model2.load_state_dict(torch.load('best_model.pth'))

In [ ]:
3.GPU加速

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()
device = torch.device('cuda')
model2 = model2.to(device)
model2 = model2.eval().half()

In [ ]:
4.图片归一化处理

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
5.显示摄像头实时画面

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
6.根据小车的行驶状况调整小车的参数

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

In [ ]:
angle = 0.0
angle_last = 0.0
x1=0
def execute(change):
    global angle, angle_last
    image = change['new']
    collision_output=collision_model(preprocesss(image)).detach().cpu()
    if x1=0:
        prob_blocked=float(F.softmax(collision_output.flatten(),dim=0)[0])
    else:
        prob_blocked=1
    if prob_blocked>0.85:
        robot.stop()
        x1=1
        return
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    if  0.05<x<=0.2:
        steering_slider.value=0.015
    elif 0.2<x<=0.25:
        steering_slider.value=0.025
    elif 0.25<x<=0.4:
        steering_slider.value=0.04
    elif 0.4<x<=1.1:
        steering_slider.value=0.05
    elif -0.2<x<=-0.05:
        steering_slider.value=-0.015
    elif -0.3<x<=-0.2:
        steering_slider.value=-0.025
    elif -0.4<x<=-0.3:
        steering_slider.value=-0.028
    elif -1.1<x<=-0.4:
        steering_slider.value=-0.04
    else:
        steering_slider.value=0
        

    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

In [ ]:
camera.observe(execute, names='value')

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()